# AUTOENCODIX PACKAGE HANDBOOK
This notebook demonstrates the usage of the autoencodix package.
For now it serves as an internal guideline with the goal to:
- test the package from a user perspective
- serve as a first draft of user documentation
- serve a developer guideline 
  - developer guide will be derrived from this notebook

## 00 Generate mock data
When  development proceeds this section should be used to  show how to use different datatypes
for now we only use a mock numpy array

In [ ]:
import numpy as np

sample_data = np.random.rand(100, 10)
sample_data.shape

(100, 10)

## 01 General Pipeline Usage

In [2]:
# imports
import autoencodix as acx
from autoencodix.utils.default_config import DefaultConfig

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1036f1750>>
Traceback (most recent call last):
  File "/Users/maximilianjoas/development/autoencodix_package/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
#### --------------------------------------------
# TODO user prepares data or config
### INITIALIZATION ### --------------------------
# Use Vanillix Pipeline interface
# needs to be initialized with data
# data should be a numpy array, pandas dataframe or AnnData object
# possible to pass a custom Config object
van = acx.Vanillix(data=sample_data)
# ------------------------------------------------
### DATA PROCESSING ### --------------------------
# job of old make data
# populates self._features attrbute with torch tensor
# populates self._datasets attribute with torch dataset
# (important for training with dataloader)
# possible to pass a custom Config object, or keyword arguments
van.preprocess()
# ------------------------------------------------
### MODEL TRAINING ### --------------------------
# job of old make model
# calls self.Trainer class to init and train model
# populates self._model attribute with trained model
# populates self.result attribute with training results (model, losses, etc)
van.fit()
# ------------------------------------------------
### PREDICTION ### -------------------------------
# job of old make predict
# if no data is passed, used the test split from preprocessing
# otherwise, uses the data passed, and preprocesses it
# updates self.result attribute with predictions (latent space, reconstructions, etc)
van.predict()
# ------------------------------------------------
### EVALUATION ### -------------------------------
# job of old make ml_task
# populates self.result attribute with ml task results
van.evaluate()  # not implemented yet
# ------------------------------------------------
### VISUALIZATION ### ---------------------------
# job of old make visualize
# populates self.result attribute with visualizations
van.visualize()
# show visualizations for notebook use
van.show_result()
# --------------------------
# --------------------------
# run all steps in the pipeline
result_object = van.run()

cpu not relevant here
batch: 0
model_outputs.reconstruction: torch.Size([32, 10])
batch: 1
model_outputs.reconstruction: torch.Size([32, 10])
batch: 2
model_outputs.reconstruction: torch.Size([6, 10])
Epoch: 0, Loss: 2.3556206822395325
output.reconstruction: torch.Size([32, 10])
output.reconstruction: torch.Size([32, 10])
output.reconstruction: torch.Size([6, 10])
output.reconstruction: torch.Size([10, 10])
batch: 0
model_outputs.reconstruction: torch.Size([32, 10])
batch: 1
model_outputs.reconstruction: torch.Size([32, 10])
batch: 2
model_outputs.reconstruction: torch.Size([6, 10])
Epoch: 1, Loss: 2.308934450149536
output.reconstruction: torch.Size([32, 10])
output.reconstruction: torch.Size([32, 10])
output.reconstruction: torch.Size([6, 10])
output.reconstruction: torch.Size([10, 10])
batch: 0
model_outputs.reconstruction: torch.Size([32, 10])
batch: 1
model_outputs.reconstruction: torch.Size([32, 10])
batch: 2
model_outputs.reconstruction: torch.Size([6, 10])
Epoch: 2, Loss: 2.2133

In [ ]:
recons = result_object.reconstructions.get(split="train", epoch=2)
recons_val = result_object.reconstructions.get(split="valid", epoch=2)
recons_test = result_object.reconstructions.get(split="test", epoch=-1)
print(recons.shape, recons_val.shape, recons_test.shape)

(70, 10) (10, 10) (20, 10)


In [ ]:
latents = result_object.latentspaces.get(split="train", epoch=2)
latents_val = result_object.latentspaces.get(split="valid", epoch=2)
latents_test = result_object.latentspaces.get(split="test", epoch=-1)
print(latents.shape, latents_val.shape, latents_test.shape)

(70, 16) (10, 16) (20, 16)


#### Using a custom train, test, valid split
When you pass the data to the pipeline, autoencodix, internally splits the data for you based on the train,test, valid ratios provided in the config (defaults are 70%/10%/20% train/valid/test).
You can either pass custom ratios (see next section) or provide the indices directly as shown below

In [ ]:
sample_data = np.random.rand(100, 10)
custom_train_indices = np.arange(75)  # we won't allow overlap between splits
custom_valid_indices = np.arange(75, 80)
custom_test_indices = np.arange(80, 100)

# the custom split needs to be a dictionary with keys "train", "valid", and "test" and indices of the samples to be included in each split as numpy arrays
custom_split = {
    "train": custom_train_indices,
    "valid": custom_valid_indices,
    "test": custom_test_indices,
}
van = acx.Vanillix(data=sample_data, custom_splits=custom_split)
van.preprocess()
van.fit(epochs=3)

cpu not relevant here


Epoch: 0, Loss: 2.367237627506256
Epoch: 1, Loss: 2.1364158391952515
Epoch: 2, Loss: 2.140314042568207


It is possible to pass empty splits, but depending on how you'll use the autoencodix pipeline, this will throw an error at some point. So it is possible to call `fit` with only training data, but if you want to call `predict` and don't provide new data, this won't work without a data in the test split

#### Using predict with new data
The standard case is to train the model with the train data and then predict with the test split.
However, it is possible to pass new data to the predict method to perform inference on this data with the already trained model

In [ ]:
new_unseen_data = np.random.rand(10, 10)
van.predict(data=new_unseen_data)


Invalid parameters: data
Valid parameters are: config


#### Examining the result of the pipeline
Each step in the pipeline writes its results in the result object of the Vanillix instance.
In this section we explore how to access and make sense of the results.

In [ ]:
result = van.result
print(result)

Result Object Public Attributes:
------------------------------
latentspaces: TrainingDynamics object
reconstructions: TrainingDynamics object
mus: TrainingDynamics object
sigmas: TrainingDynamics object
losses: TrainingDynamics object
preprocessed_data: Tensor of shape (100, 10)
model: _FabricModule
model_checkpoints: TrainingDynamics object
datasets: DatasetContainer(train=<autoencodix.data._numeric_dataset.NumericDataset object at 0x1053f1e70>, valid=<autoencodix.data._numeric_dataset.NumericDataset object at 0x10626cd90>, test=<autoencodix.data._numeric_dataset.NumericDataset object at 0x10626ce20>)


##### TrainingDynamics object in result
The training dynamics object has the followinf form:
<epoch><split><data>
So if you want to access the train loss for the 5th epoch, you would:
`result.lossss.get(epoch=5, split="train")`

In [ ]:
loss_train_ep2 = result.losses.get(epoch=2, split="train")
print(loss_train_ep2)
valid_loss = result.losses.get(split="valid")
print(valid_loss)
print(result.losses.get())

0.7134380141894022
[0.27492604 0.24243712 0.23720634]
{0: {'train': array(0.78907921), 'valid': array(0.27492604)}, 1: {'train': array(0.71213861), 'valid': array(0.24243712)}, 2: {'train': array(0.71343801), 'valid': array(0.23720634)}}


Note: this schema works for every TrainingDynamics instance in the results object.

## 02 Pipeline usage with custom parameters
Here we show how to customize the above shown pipeline with a user config or with keyword arguments.
In future iterations we want to allow to read a config from a file, this will be also demonstrated here.

In [ ]:
# Use Vanillix Pipeline interface
# needs to be initialized with data
# data should be a numpy array, pandas dataframe or AnnData object
# possible to pass a custom Config object
van = acx.Vanillix(data=sample_data)
# job of old make data
# populates self._features attrbute with torch tensor
# populates self._datasets attribute with torch dataset
# (important for training with dataloader)
# possible to pass a custom Config object, or keyword arguments
van.preprocess()
# job of old make model
# calls self.Trainer class to init and train model
# populates self._model attribute with trained model
# populates self.result attribute with training results (losses, etc)
# van.fit()
""" 
Each step can be run separately, with custom parameters, these parameters
can be passed as keyword arguments, or as a Config object
"""
van.fit(learning_rate=0.01, batch_size=32, epochs=5)  # or like this:
my_config = DefaultConfig(learning_rate=130.0, batch_size=32, epochs=5)
van.fit(config=my_config)  # config has to be an keyword argument


cpu not relevant here
Epoch: 0, Loss: 1.9614873230457306
Epoch: 1, Loss: 1.382771223783493
Epoch: 2, Loss: 0.977891355752945
Epoch: 3, Loss: 0.6883653551340103
Epoch: 4, Loss: 0.5179779827594757
cpu not relevant here
Epoch: 0, Loss: 8780462592.772186
Epoch: 1, Loss: 1348435297.7003174
Epoch: 2, Loss: 5917.90837097168
Epoch: 3, Loss: 10487.504081726074
Epoch: 4, Loss: 3092.8131675720215


#### 02.1  How to relevant keyword arguments for pipeline methods
It can be hard to know what keyword arguments are valid for each step,
so we show:
- how to get a list of allowed keyword arguments
- what happens if you pass non-allowed keyword arguments

In [ ]:
# for each config method, we can call a valid_params method
van = acx.Vanillix(data=sample_data)
fit_params = (
    van.fit.valid_params
)  # returns a set of keyword arguments that are actually used in the fit method

import pprint

pprint.pprint(fit_params)

{'batch_size',
 'checkpoint_interval',
 'config',
 'device',
 'epochs',
 'global_seed',
 'gpu_strategy',
 'learning_rate',
 'n_gpus',
 'n_workers',
 'reconstruction_loss',
 'reproducible',
 'weight_decay'}


To get even more verbose info about the keyword args, you can run the following code.

In [ ]:
# when you want to have more info about the params, you can get type hints from the config object
my_config = DefaultConfig()
conig_values = my_config.get_params()
my_config.print_schema(filter_params=fit_params)

Valid Keyword Arguments:
--------------------------------------------------

learning_rate:
  Type: <class 'float'>
  Default: 0.001
  Description: Learning rate for optimization

batch_size:
  Type: <class 'int'>
  Default: 32
  Description: Number of samples per batch

epochs:
  Type: <class 'int'>
  Default: 3
  Description: Number of training epochs

weight_decay:
  Type: <class 'float'>
  Default: 0.01
  Description: L2 regularization factor

reconstruction_loss:
  Type: typing.Literal['mse', 'bce']
  Default: mse
  Description: Type of reconstruction loss

device:
  Type: typing.Literal['cpu', 'cuda', 'gpu', 'tpu', 'mps', 'auto']
  Default: auto
  Description: Device to use

n_gpus:
  Type: <class 'int'>
  Default: 1
  Description: Number of GPUs to use

n_workers:
  Type: <class 'int'>
  Default: 2
  Description: Number of data loading workers

checkpoint_interval:
  Type: <class 'int'>
  Default: 1
  Description: Interval for saving checkpoints

gpu_strategy:
  Type: typing.Lit

If you pass not supported parameters you get a warning

In [ ]:
# if you use an unsupported keyword argument, you will get a warning
# as you see the default value from the DefaultConfig is not overwritten and the training will take 100 epochs (not 10)
van.preprocess()
van.fit(epochds=10)


Invalid parameters: epochds
Valid parameters are: batch_size, checkpoint_interval, config, device, epochs, global_seed, gpu_strategy, learning_rate, n_gpus, n_workers, reconstruction_loss, reproducible, weight_decay
cpu not relevant here
Epoch: 0, Loss: 2.2837421894073486
Epoch: 1, Loss: 2.2954598665237427
Epoch: 2, Loss: 2.3972206711769104


#### 02.2 How to get information about the default config parameters

In [ ]:
# if you want to see what config parameters are used in the default config you can do it like:
default_config = DefaultConfig()
default_config.print_schema()



DefaultConfig Configuration Parameters:
--------------------------------------------------

latent_dim:
  Type: <class 'int'>
  Default: 16
  Description: Dimension of the latent space

n_layers:
  Type: <class 'int'>
  Default: 3
  Description: Number of layers in encoder/decoder

enc_factor:
  Type: <class 'int'>
  Default: 4
  Description: Scaling factor for encoder dimensions

input_dim:
  Type: <class 'int'>
  Default: 10000
  Description: Input dimension

drop_p:
  Type: <class 'float'>
  Default: 0.1
  Description: Dropout probability

learning_rate:
  Type: <class 'float'>
  Default: 0.001
  Description: Learning rate for optimization

batch_size:
  Type: <class 'int'>
  Default: 32
  Description: Number of samples per batch

epochs:
  Type: <class 'int'>
  Default: 3
  Description: Number of training epochs

weight_decay:
  Type: <class 'float'>
  Default: 0.01
  Description: L2 regularization factor

reconstruction_loss:
  Type: typing.Literal['mse', 'bce']
  Default: mse
  

### 02.3 Documentation Config class
You can update the config with your own values by:
- passing arguments as:
    - dict
    - single arguments
- passing a file (TODO)

In [ ]:
from autoencodix.utils.default_config import DefaultConfig

# METHOD 1: override the default config with a dictionary
my_args = {"learning_rate": 0.0234, "batch_size": 13, "epochs": 12}
my_config = DefaultConfig(**my_args)
# METHOD 2: override signle parameters
my_new_conig = DefaultConfig(latent_dim=23, n_gpus=13)

# METHOD 3: from a file: TODO


## 03 Use the Varix model
Now we show how easy it is to use a variational autoencoder instead of a vanilla version.

In [7]:
from autoencodix.utils.default_config import DefaultConfig
import autoencodix as acx
import numpy as np

sample_data = np.random.rand(100, 10)
my_config = DefaultConfig(learning_rate=0.001, epochs=3, checkpoint_interval=1)
varix = acx.Varix(data=sample_data, config=my_config)
result = varix.run()

cpu not relevant here
Epoch: 0, Loss: 1.3720149099826813
Epoch: 1, Loss: 1.1976171135902405
Epoch: 2, Loss: 1.221748024225235

Invalid parameters: data
Valid parameters are: config


#### Examine Variational result
Here, we have more info in our results object than in the Vanillix case. We have the learned paramters mu and logvar of the normal distirbution, in addition to the losses and reconstructions. We provide also the sampled latentspaces at each epoch and split.

You can resample new latenspaces (shown in next section)

In [6]:
# we did not train for the test split, so we don't need to pass an epoch
# technically the epoch is -1
mu_test_ep_last = result.latentspaces.get(split="test")
print(mu_test_ep_last.shape)

(1, 20, 16)


#### Different loss types
For our variation autoencoder, the total loss consists of a reconstruction loss and a distribution loss i.e. kl-divergence. To investigate these losses, the result_obj has the attribute `sub_losses`. This is a `LossRegistry` withe the name of the loss as key and the value is of class `TrainingDynamics` and can be accessed as shown for the Vanillix part

In [7]:
sub_losses = result.sub_losses
print(f"keys: {sub_losses.keys()}")
recon_dyn = sub_losses.get(key="recon_loss")
print(recon_dyn.get(split="train"))

keys: dict_keys(['recon_loss', 'var_loss'])
[0.42741003 0.44090185 0.37895077]


#### Sample new latentspaces
You might want to use the trained model and the fitted parameters mu, and logvar to sample latentspaces. Therefore, the Varix pipeline has the additional method `sample_latent_space`

In [6]:
sampled = varix.sample_latent_space()

print(sampled[:5])

tensor([[ 1.4227, -1.1264, -0.5146, -0.6423,  1.3555,  0.1267,  1.1256, -0.8374,
          1.2268, -1.7507,  0.9605, -0.4691,  0.9137, -0.8082, -0.5185, -0.1135],
        [ 0.0871, -0.7650, -0.7087,  1.9782,  0.5053,  0.0163,  0.5845, -0.4165,
          0.2073,  1.6614, -0.8132,  0.6487, -1.5413, -0.4115,  0.0680,  0.1220],
        [ 0.5332,  0.1249,  1.5426,  1.3922,  0.3982,  0.5452,  0.5251, -0.1709,
          0.2392, -0.6901,  0.6993, -2.4067,  0.7687,  0.4694, -1.3835, -1.3028],
        [ 0.4925,  0.2083, -0.3846,  0.7595, -0.3062, -0.2996, -0.4743,  1.3784,
          1.0943, -0.0518, -0.3465, -0.3751,  0.3677,  0.3754, -0.2108, -0.4754],
        [-0.1330,  1.1649,  0.2671,  0.2378, -3.0322, -1.7903, -0.1169,  1.0738,
         -0.9661,  0.0313,  0.2592, -1.3385,  0.7877, -0.0342,  1.4808,  2.4948]],
       device='mps:0')


In [8]:
# you can also select a specific epoch and split to sample from (default is last epoch and test split)
sampled = varix.sample_latent_space(epoch=2, split="valid")
print(sampled[:5])

tensor([[ 0.3581,  0.8946,  2.3437, -1.0793,  1.5991, -0.6819,  1.3511,  0.7896,
         -0.6627,  1.4723,  1.3740, -0.8405, -0.3541, -0.3976,  1.8037, -0.3309],
        [-2.7742,  1.3001, -0.7447, -1.5788, -1.3535,  0.3174, -0.5517,  0.9802,
         -1.0824,  1.4315, -0.8219, -1.6757,  0.4232, -1.0295, -1.2279,  0.2959],
        [-0.5726,  1.2144,  1.3754, -0.1848,  1.2447, -0.2668, -1.7038,  1.4772,
          0.6938,  0.8111,  0.5638, -1.3457,  1.3015, -2.0530, -1.4406, -0.6207],
        [-0.5844,  1.0119,  1.3542,  0.8185,  0.4677,  0.6580, -0.4376, -1.5302,
          0.3264,  0.5532,  1.9664, -1.2441,  1.4664, -1.4824,  1.2571, -1.7927],
        [ 1.2330, -0.1874, -0.8378,  1.5088, -0.8935, -0.0678, -0.0589, -0.2272,
          0.4520, -1.1687,  0.4746,  0.1345, -0.5463, -0.4620,  0.9897, -0.3327]],
       device='mps:0')


In [10]:
# or sample multiple times
for _ in range(5):
    sampled = varix.sample_latent_space()
    print(sampled[:3, :3])

tensor([[ 0.8121, -0.2599, -1.1656],
        [-1.5269,  0.5212, -0.3416],
        [ 1.1872,  1.8373,  0.4524]], device='mps:0')
tensor([[ 2.3787,  0.1305, -0.7975],
        [-0.3502, -0.2842,  1.0856],
        [-2.3673,  1.2124,  1.5476]], device='mps:0')
tensor([[ 0.0609,  2.3645,  0.2055],
        [-0.3573, -1.5928,  0.5545],
        [-2.5494, -0.6906, -0.8733]], device='mps:0')
tensor([[ 0.9513,  1.5320,  1.5767],
        [ 0.4684, -1.0381,  0.9011],
        [-1.0837,  1.9147,  0.4768]], device='mps:0')
tensor([[-0.0046, -1.3439, -0.2800],
        [ 1.0443, -0.2889, -0.1052],
        [ 0.5079,  1.7101,  0.0815]], device='mps:0')


## How to add a new architecture
If you want to add a new architecture


# TODOS
- show how to update and work with the config object (later)

### SANDBOX 
testing Varix and losses, especially sub_losses in result_object

In [2]:
varix._trainer._model

_FabricModule(
  (_forward_module): VarixArchitecture(
    (_encoder): Sequential(
      (0): Linear(in_features=10, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=16, bias=True)
      (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplace=False)
      (7): ReLU()
      (8): Linear(in_features=16, out_features=16, bias=True)
    )
    (_decoder): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=16, bias=True)
      (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplac

In [2]:
sub_losses = result.sub_losses

In [2]:
from autoencodix.utils._result import Result, LossRegistry
from autoencodix.utils.default_config import DefaultConfig
from autoencodix.utils._traindynamics import TrainingDynamics

In [ ]:
empty_result = Result()
sample_model = None
sample_datasets = {"train": None, "valid": None, "test": None}
sample_recon_data = {
    0: {"train": 0.0, "valid": 0.0, "test": 0.0},
    100: {"train": 0.0, "valid": 0.0, "test": 0.0},
}
sample_var_loss_data = {
    0: {"train": 0.0, "valid": 0.0, "test": 0.0},
    2: {"train": 0.0, "valid": 0.0, "test": 0.0},
    100: {"train": 0.0, "valid": 0.0, "test": 0.0},
}
sample_recon_data_dyn = TrainingDynamics(_data=sample_recon_data)
sample_var_loss_data_dyn = TrainingDynamics(_data=sample_var_loss_data)
sample_losses = LossRegistry(
    _losses={"recon_loss": sample_recon_data_dyn, "var_loss": sample_var_loss_data_dyn}
)
filled_result = Result(
    model=sample_model, datasets=sample_datasets, sub_losses=sample_losses
)

recon_loss_data1 = {
    0: {"valid": 0.1, "train": 0.2, "test": 0.3},
    1: {"valid": 0.4, "train": 0.5, "test": 0.6},
    2: {"valid": 0.7, "train": 0.8, "test": 0.9},
}
var_loss_data1 = {
    0: {"valid": 0.11, "train": 0.21, "test": 0.31},
    1: {"valid": 0.31, "train": 0.41, "test": 0.51},
    2: {"valid": 0.51, "train": 0.61, "test": 0.71},
}

recon_dynamics1 = TrainingDynamics(_data=recon_loss_data1)
var_dynamics1 = TrainingDynamics(_data=var_loss_data1)
sample_registry = LossRegistry(
    _losses={"recon_loss": recon_dynamics1, "var_loss": var_dynamics1}
)
sample_result1 = Result(sub_losses=sample_registry)
recon_loss_data2 = {
    0: {"valid": 0.15, "train": 0.25},
    1: {"valid": 0.45, "train": 0.55, "test": None},
    2: {"valid": None, "train": 0.85, "test": 0.95},
}
var_loss_data2 = {
    0: {"valid": 0.16, "train": 0.26, "test": 0.36},
    1: {"valid": 0.36, "train": 0.46},
    2: None,
}  # should not be updated

recon_dynamics2 = TrainingDynamics(_data=recon_loss_data2)
var_dynamics2 = TrainingDynamics(_data=var_loss_data2)
sample_registry2 = LossRegistry(
    _losses={"recon_loss": recon_dynamics2, "var_loss": var_dynamics2}
)
sample_result2 = Result(sub_losses=sample_registry2)

In [16]:
to_update = {
    0: {"train": 0.0, "valid": 0.0, "test": 0.0},  # totally overwrite
    2: {"train": 0.0, "valid": 0.0, "test": 0.0},  # keep all
    3: {},  # update with partial data
    4: None,  # udate with all
    5: {
        "train": None,
        "valid": 0.0,
        "test": 0.0,
    },  # update with partial data (keep test)
    6: {"train": 0.0},  # update with partial data (keep train), add vlaid and test
    7: {"train": 0},  # update with partial data {train: 0.1}
    100: {"train": 0.0, "valid": 0.0, "test": 0.0},  # not in to update
}

update_with = {
    0: {"train": 0.1, "valid": 0.2, "test": 0.3},
    1: {"train": 0.4, "valid": 0.5, "test": 0.6},
    2: {},
    3: {"train": 0.7, "valid": 0.8},
    4: {"train": 0.9, "valid": 1.0, "test": 1.1},
    5: {"train": 0.12, "valid": 0.22},
    6: {"valid": 0.32, "test": 0.42},
    7: {"train": 0.1},
}

expected_result = {
    0: {"train": np.array(0.1), "valid": np.array(0.2), "test": np.array(0.3)},
    1: {"train": np.array(0.4), "valid": np.array(0.5), "test": np.array(0.6)},
    2: {"train": np.array(0.0), "valid": np.array(0.0), "test": np.array(0.0)},
    3: {"train": np.array(0.7), "valid": np.array(0.8)},
    4: {"train": np.array(0.9), "valid": np.array(1.0), "test": np.array(1.1)},
    5: {"train": np.array(0.12), "valid": np.array(0.22), "test": np.array(0.0)},
    6: {"train": np.array(0.0), "valid": np.array(0.32), "test": np.array(0.42)},
    7: {"train": np.array(0.1)},
    100: {"train": np.array(0.0), "valid": np.array(0.0), "test": np.array(0.0)},
}

to_update_dyn = TrainingDynamics(_data=to_update)
to_update_registry = LossRegistry(
    _losses={
        "recon_loss": to_update_dyn,
        "var_loss": to_update_dyn,
        "more_loss": TrainingDynamics(_data={}),
    }
)
to_update_result = Result(sub_losses=to_update_registry)


update_with_dyn = TrainingDynamics(_data=update_with)
update_with_registry = LossRegistry(
    _losses={
        "recon_loss": update_with_dyn,
        "var_loss": TrainingDynamics(_data={}),
        "other_loss": TrainingDynamics(_data={}),
    }
)
update_with_result = Result(sub_losses=update_with_registry)
to_update_result.update(update_with_result)
after_update = dict(
    sorted(to_update_result.sub_losses.get(key="recon_loss").get().items())
)
expected_recon_dyn = TrainingDynamics(_data=expected_result)
expected_varloss_dyn = TrainingDynamics(_data=to_update)
expected_moreloss_dyn = TrainingDynamics(_data={})
expected_other_dyn = TrainingDynamics(_data={})
expected_loss_registry = LossRegistry(
    _losses={
        "recon_loss": expected_recon_dyn,
        "var_loss": expected_varloss_dyn,
        "more_loss": expected_moreloss_dyn,
        "other_loss": expected_other_dyn,
    }
)
expected_result = Result(sub_losses=expected_loss_registry)


In [17]:

to_update_result.sub_losses._losses
for lossname, dynamics in expected_result.sub_losses.losses():
    print(lossname)
    print(type(dynamics._data))

recon_loss
<class 'dict'>
var_loss
<class 'dict'>
more_loss
<class 'dict'>
other_loss
<class 'dict'>


In [21]:
for lossname, dynamics in to_update_result.sub_losses.losses():
    print(lossname)
    print(dynamics.get())

recon_loss
{0: {'train': array(0.1), 'valid': array(0.2), 'test': array(0.3)}, 1: {'train': array(0.4), 'valid': array(0.5), 'test': array(0.6)}, 2: {'train': array(0.), 'valid': array(0.), 'test': array(0.)}, 3: {'valid': array(0.8), 'train': array(0.7)}, 4: {'valid': array(1.), 'test': array(1.1), 'train': array(0.9)}, 5: {'valid': array(0.22), 'test': array(0.), 'train': array(0.12)}, 6: {'train': array(0.), 'valid': array(0.32), 'test': array(0.42)}, 7: {'train': array(0.1)}, 100: {'train': array(0.), 'valid': array(0.), 'test': array(0.)}}
var_loss
{0: {'train': array(0.), 'valid': array(0.), 'test': array(0.)}, 2: {'train': array(0.), 'valid': array(0.), 'test': array(0.)}, 5: {'valid': array(0.), 'test': array(0.)}, 6: {'train': array(0.)}, 7: {'train': array(0)}, 100: {'train': array(0.), 'valid': array(0.), 'test': array(0.)}}
more_loss
{}
other_loss
{}


In [22]:
for lossname, dynamics in expected_result.sub_losses.losses():
    print(lossname)
    print(dynamics.get())

recon_loss
{0: {'train': array(0.1), 'valid': array(0.2), 'test': array(0.3)}, 1: {'train': array(0.4), 'valid': array(0.5), 'test': array(0.6)}, 2: {'train': array(0.), 'valid': array(0.), 'test': array(0.)}, 3: {'train': array(0.7), 'valid': array(0.8)}, 4: {'train': array(0.9), 'valid': array(1.), 'test': array(1.1)}, 5: {'train': array(0.12), 'valid': array(0.22), 'test': array(0.)}, 6: {'train': array(0.), 'valid': array(0.32), 'test': array(0.42)}, 7: {'train': array(0.1)}, 100: {'train': array(0.), 'valid': array(0.), 'test': array(0.)}}
var_loss
{0: {'train': 0.0, 'valid': 0.0, 'test': 0.0}, 2: {'train': 0.0, 'valid': 0.0, 'test': 0.0}, 3: {}, 4: None, 5: {'train': None, 'valid': 0.0, 'test': 0.0}, 6: {'train': 0.0}, 7: {'train': 0}, 100: {'train': 0.0, 'valid': 0.0, 'test': 0.0}}
more_loss
{}
other_loss
{}


In [ ]:
(filled_result.sub_losses._losses["var_loss"].get())

{0: {'train': array(0.26), 'valid': array(0.16), 'test': array(0.36)},
 2: {'train': 0.0, 'valid': 0.0, 'test': 0.0},
 100: {'train': 0.0, 'valid': 0.0, 'test': 0.0},
 1: {'valid': array(0.36), 'train': array(0.46)}}

In [16]:
# empty does not overwrite anything
result.update(empty_result)
result.losses.get()
result.datasets

DatasetContainer(train=<autoencodix.data._numeric_dataset.NumericDataset object at 0x15ff96b30>, valid=<autoencodix.data._numeric_dataset.NumericDataset object at 0x15ff96bc0>, test=<autoencodix.data._numeric_dataset.NumericDataset object at 0x15ff96e00>)

In [10]:
# case one update empty result
# with: - empty result, - sample result1, - sample result2
empty_result.update(other=sample_result1)
empty_result.sub_losses._losses["recon_loss"].get()


{0: {'valid': array(0.1), 'train': array(0.2), 'test': array(0.3)},
 1: {'valid': array(0.4), 'train': array(0.5), 'test': array(0.6)},
 2: {'valid': array(0.7), 'train': array(0.8), 'test': array(0.9)}}

In [6]:
empty_result.update(other=sample_result2)

In [7]:
# case two update filled result
# with: - filled result, - sample result1, - sample result2, empty result
filled_result.update(other=sample_result1)

In [6]:
sub_losses == updated_losses

True

In [5]:
updated_losses = result.sub_losses
print(updated_losses)

LossRegistry(_losses={'recon_loss': TrainingDynamics(), 'var_loss': TrainingDynamics()})


In [9]:
for name, loss in updated_losses.losses():
    print(f"{name}: {loss}")
    print(loss.get())

recon_loss: TrainingDynamics()
{0: {'train': array(0.44560597), 'valid': array(0.24305786)}, 1: {'train': array(0.40050308), 'valid': array(0.23968849)}, 2: {'train': array(0.39850321), 'valid': array(0.2438124)}}
var_loss: TrainingDynamics()
{0: {'train': array(0.03078574), 'valid': array(0.00519827)}, 1: {'train': array(0.02884311), 'valid': array(0.00536202)}, 2: {'train': array(0.02645038), 'valid': array(0.00561051)}}


In [4]:
varix._trainer._model

_FabricModule(
  (_forward_module): VarixArchitecture(
    (_encoder): Sequential(
      (0): Linear(in_features=10, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=16, bias=True)
      (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplace=False)
      (7): ReLU()
      (8): Linear(in_features=16, out_features=16, bias=True)
    )
    (_decoder): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=16, bias=True)
      (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplac

In [5]:
DefaultConfig().print_schema()


DefaultConfig Configuration Parameters:
--------------------------------------------------

latent_dim:
  Type: <class 'int'>
  Default: 16
  Description: Dimension of the latent space

n_layers:
  Type: <class 'int'>
  Default: 3
  Description: Number of layers in encoder/decoder

enc_factor:
  Type: <class 'int'>
  Default: 4
  Description: Scaling factor for encoder dimensions

input_dim:
  Type: <class 'int'>
  Default: 10000
  Description: Input dimension

drop_p:
  Type: <class 'float'>
  Default: 0.1
  Description: Dropout probability

learning_rate:
  Type: <class 'float'>
  Default: 0.001
  Description: Learning rate for optimization

batch_size:
  Type: <class 'int'>
  Default: 32
  Description: Number of samples per batch

epochs:
  Type: <class 'int'>
  Default: 3
  Description: Number of training epochs

weight_decay:
  Type: <class 'float'>
  Default: 0.01
  Description: L2 regularization factor

reconstruction_loss:
  Type: typing.Literal['mse', 'bce']
  Default: mse
  

In [6]:
config = DefaultConfig()
config.n_layers

3

### Check model architecture layers

varix.

In [8]:
varix._trainer._model

_FabricModule(
  (_forward_module): VarixArchitecture(
    (_encoder): Sequential(
      (0): Linear(in_features=10, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=16, bias=True)
      (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplace=False)
      (7): ReLU()
      (8): Linear(in_features=16, out_features=16, bias=True)
    )
    (_decoder): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=16, bias=True)
      (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplac

In [9]:
config = DefaultConfig()
config.n_layers

3

In [14]:
from autoencodix.trainers._general_trainer import GeneralTrainer
import autoencodix as acx
from autoencodix.utils.default_config import DefaultConfig
import numpy as np
sample_data = np.random.rand(10, 100)

varix = acx.Varix(data=sample_data, config=DefaultConfig(n_layers=3, epochs=1, enc_factor=2,latent_dim=2))
varix.preprocess()
varix.fit()
print(varix._trainer._model)
for enc_factor in [1, 2, 3]:
    print(f"ENCODER FACTOR: {enc_factor}")
    varix = acx.Varix(data=sample_data, config=DefaultConfig(n_layers=2, epochs=1, enc_factor=enc_factor))
    varix.preprocess()
    varix.fit()
    print(varix._trainer._model)

cpu not relevant here
[100, 50, 25, 12, 2]
Epoch: 0, Loss: 0.5027862191200256
_FabricModule(
  (_forward_module): VarixArchitecture(
    (_mu): Linear(in_features=12, out_features=2, bias=True)
    (_logvar): Linear(in_features=12, out_features=2, bias=True)
    (_encoder): Sequential(
      (0): Linear(in_features=100, out_features=50, bias=True)
      (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=50, out_features=25, bias=True)
      (5): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.1, inplace=False)
      (7): ReLU()
      (8): Linear(in_features=25, out_features=12, bias=True)
      (9): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): Dropout(p=0.1, inplace=False)
      (11): ReLU()
    )
    (_decoder): Sequential(
      (0): Linear(in_features=2, out_fea

In [13]:
len(varix._trainer._model._decoder)

1

In [15]:
import torch.nn as nn
empty = nn.Sequential()

In [16]:
len(empty)

0

In [4]:
van = acx.Vanillix(data=sample_data, config=DefaultConfig(n_layers=0, epochs=1))
van.preprocess()
van.fit()
print(van._trainer._model)

cpu not relevant here


Epoch: 0, Loss: 0.5317338109016418
_FabricModule(
  (_forward_module): VanillixArchitecture(
    (_encoder): Sequential(
      (0): Linear(in_features=100, out_features=16, bias=True)
    )
    (_decoder): Sequential(
      (0): Linear(in_features=16, out_features=100, bias=True)
    )
  )
  (_original_module): VanillixArchitecture(
    (_encoder): Sequential(
      (0): Linear(in_features=100, out_features=16, bias=True)
    )
    (_decoder): Sequential(
      (0): Linear(in_features=16, out_features=100, bias=True)
    )
  )
)
